In [7]:
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import pickle
import pandas as pd
import gdown
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from lightgbm import LGBMRegressor


# save dataset

In [14]:
train_dtypes = {
    'molecule_name': 'category',
    'atom_index_0': 'int8',
    'atom_index_1': 'int8',
    'type': 'category',
    'scalar_coupling_constant': 'float32'
}
# Se define un diccionario de números atómicos para elementos químicos
ATOMIC_NUMBERS = {
    'H': 1,  #Hidrógeno
    'C': 6,  #Carbono
    'N': 7,  #Nitrógeno
    'O': 8,  #Oxígeno
    'F': 9   #Fluor
}
#Mostrar cadenas de texto largas sin truncarlas
pd.set_option('display.max_colwidth', None)
#mostrar hasta 120 filas antes de truncar la salida
pd.set_option('display.max_rows', 120)
#mostrar hasta 120 columnas antes de truncar la salida
pd.set_option('display.max_columns', 120)
#Dataframe magnetic shielding tensors
df_train_sub_tensor = pd.read_csv('magnetic_shielding_tensors.csv', delimiter = ';')

In [15]:
#Dataframe mulliken charges
df_train_sub_charge = pd.read_csv('mulliken_charges.csv', delimiter = ';')

In [17]:
#Dataframe structures
structures_dtypes = {
    'molecule_name': 'category',
    'atom_index': 'int8',
    'atom': 'category',
    'x': 'float32',
    'y': 'float32',
    'z': 'float32'
}

structures_csv = pd.read_csv('structures.csv', dtype=structures_dtypes)
structures_csv['molecule_index'] = structures_csv.molecule_name.str.replace('dsgdb9nsd_', '').astype('int32')
structures_csv = structures_csv[['molecule_index', 'atom_index', 'atom', 'x', 'y', 'z']]

structures_csv['atom'] = structures_csv['atom'].astype(str)
structures_csv['atom'] = structures_csv['atom'].replace(ATOMIC_NUMBERS).astype('int8')


In [18]:
#Dataframe test. Conjunto de prueba
test_csv = pd.read_csv('test.csv', index_col='id', dtype=train_dtypes)

#Añadir columna 'molecule_index'. Identificador numérico para cada molécula
test_csv['molecule_index'] = test_csv['molecule_name'].str.replace('dsgdb9nsd_', '').astype('int32')

#Selección del subconjunto de columnas para conservar en el DataFrame `test_csv`.
test_csv = test_csv[['molecule_index', 'atom_index_0', 'atom_index_1', 'type']]
test_csv.head(10)

,molecule_index,atom_index_0,atom_index_1,type
id,,,,
4659076,4,2,0,2JHC
4659077,4,2,1,1JHC
4659078,4,2,3,3JHH
4659079,4,3,0,1JHC
4659080,4,3,1,2JHC
4659081,14,3,0,1JHC
4659082,14,3,1,2JHC
4659083,14,3,4,2JHH
4659084,14,3,5,2JHH


In [19]:
#Dataframe sample.

submission_csv = pd.read_csv('sample_submission.csv', index_col='id')
print(submission_csv.head())

         scalar_coupling_constant
id                               
4659076                       0.0
4659077                       0.0
4659078                       0.0
4659079                       0.0
4659080                       0.0


In [21]:
# Ver las primeras líneas del archivo CSV para verificar su contenido
with open('train.csv', 'r') as file:
    for _ in range(5):
        print(file.readline())


id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant

0,dsgdb9nsd_000001,1,0,1JHC,84.8076

1,dsgdb9nsd_000001,1,2,2JHH,-11.257

2,dsgdb9nsd_000001,1,3,2JHH,-11.2548

3,dsgdb9nsd_000001,1,4,2JHH,-11.2543



In [36]:
#Dataframe train
# driveUrlTrain = 'https://drive.google.com/uc?id=1cFjgpNoWLIFtvfBKqsxeZ_vWNeuelhwh'
# outputFileTrain = 'train.csv'
# gdown.download(driveUrlTrain, outputFileTrain, quiet=False)
train_csv = pd.read_csv('train.csv', index_col='id', dtype=train_dtypes)
train_csv['molecule_index'] = train_csv.molecule_name.str.replace('dsgdb9nsd_', '').astype('int32')
train_csv = train_csv[['molecule_index', 'atom_index_0', 'atom_index_1', 'type', 'scalar_coupling_constant']]
train_csv.head(10)
# train_csv = pd.read_csv('train.csv', index_col='id')
# train_csv['molecule_index'] = train_csv.molecule_name.str.replace('dsgdb9nsd_', '').astype('int32')
# train_csv = train_csv[['molecule_index', 'atom_index_0', 'atom_index_1', 'type', 'scalar_coupling_constant']]

,molecule_index,atom_index_0,atom_index_1,type,scalar_coupling_constant
id,,,,,
0,1,1,0,1JHC,84.807602
1,1,1,2,2JHH,-11.257000
2,1,1,3,2JHH,-11.254800
3,1,1,4,2JHH,-11.254300
4,1,2,0,1JHC,84.807404
5,1,2,3,2JHH,-11.254100
6,1,2,4,2JHH,-11.254800
7,1,3,0,1JHC,84.809303
8,1,3,4,2JHH,-11.254300


In [50]:
#Función para filtrar por tipo de acopamiento dado

def build_type_dataframes(base, structures, coupling_type):
    base = base[base['type'] == coupling_type].drop('type', axis=1).copy()
    base = base.reset_index()
    base['id'] = base['id'].astype('int32')
    structures = structures[structures['molecule_index'].isin(base['molecule_index'])]
    return base, structures

#Función para tomar un conjunto base de datos (base), combinarlo con un conjunto de estructuras moleculares (structures) para añadir las coordenadas (x, y, z) de los átomos específicos
def add_coordinates(base, structures, index):
    df = pd.merge(base, structures, how='inner',
                  left_on=['molecule_index', f'atom_index_{index}'],
                  right_on=['molecule_index', 'atom_index']).drop(['atom_index'], axis=1)
    df = df.rename(columns={
        'atom': f'atom_{index}',
        'x': f'x_{index}',
        'y': f'y_{index}',
        'z': f'z_{index}'
    })
    return df

#Función para fusionar los DataFrames base y atoms en función de las columnas 'molecule_index', 'atom_index_0', y 'atom_index_1'
def add_atoms(base, atoms):
    df = pd.merge(base, atoms, how='inner',
                  on=['molecule_index', 'atom_index_0', 'atom_index_1'])
    return df

#Función para combinar los DataFrames base y structures  y luego filtrar las filas donde los índices de átomos coinciden
def merge_all_atoms(base, structures):
    df = pd.merge(base, structures, how='left',
                  left_on=['molecule_index'],
                  right_on=['molecule_index'])
    df = df[(df.atom_index_0 != df.atom_index) & (df.atom_index_1 != df.atom_index)]
    return df

#Función para calcular el centro entre dos puntos de coordenadas espaciales de átomos
def add_center(df):
    df['x_c'] = ((df['x_1'] + df['x_0']) * np.float32(0.5))
    df['y_c'] = ((df['y_1'] + df['y_0']) * np.float32(0.5))
    df['z_c'] = ((df['z_1'] + df['z_0']) * np.float32(0.5))

#Función para calcular la distancia euclidiana entre cada punto representado por las coordenadas (x, y, z)

def add_distance_to_center(df):
    df['d_c'] = ((
        (df['x_c'] - df['x'])**np.float32(2) +
        (df['y_c'] - df['y'])**np.float32(2) +
        (df['z_c'] - df['z'])**np.float32(2)
    )**np.float32(0.5))

#Función para calcular la distancia euclidiana entre dos puntos especificados por los sufijos de columnas dados (suffix1 y suffix2)
def add_distance_between(df, suffix1, suffix2):
    df[f'd_{suffix1}_{suffix2}'] = ((
        (df[f'x_{suffix1}'] - df[f'x_{suffix2}'])**np.float32(2) +
        (df[f'y_{suffix1}'] - df[f'y_{suffix2}'])**np.float32(2) +
        (df[f'z_{suffix1}'] - df[f'z_{suffix2}'])**np.float32(2)
    )**np.float32(0.5))

#Función para calcular todas las distancias entre pares de átomos en el DataFrame
def add_distances(df):
    n_atoms = 1 + max([int(c.split('_')[1]) for c in df.columns if c.startswith('x_')])

    for i in range(1, n_atoms):
        for vi in range(min(4, i)):
            add_distance_between(df, i, vi)

#Función para agregar información sobre el número de átomos de cada molécula al DataFrame base
def add_n_atoms(base, structures):
    dfs = structures['molecule_index'].value_counts().rename('n_atoms').to_frame()
    return pd.merge(base, dfs, left_on='molecule_index', right_index=True)

#Función para construir un dataframe con características específicas de tipo de acoplamiento químico
def build_couple_dataframe(some_csv, structures_csv, coupling_type, n_atoms=10):
    base, structures = build_type_dataframes(some_csv, structures_csv, coupling_type)
    base = add_coordinates(base, structures, 0)
    base = add_coordinates(base, structures, 1)

    base = base.drop(['atom_0', 'atom_1'], axis=1)
    atoms = base.drop('id', axis=1).copy()
    if 'scalar_coupling_constant' in some_csv:
        atoms = atoms.drop(['scalar_coupling_constant'], axis=1)

    add_center(atoms)
    atoms = atoms.drop(['x_0', 'y_0', 'z_0', 'x_1', 'y_1', 'z_1'], axis=1)

    atoms = merge_all_atoms(atoms, structures)
    add_distance_to_center(atoms)

    atoms = atoms.drop(['x_c', 'y_c', 'z_c', 'atom_index'], axis=1)
    atoms.sort_values(['molecule_index', 'atom_index_0', 'atom_index_1', 'd_c'], inplace=True)
    atom_groups = atoms.groupby(['molecule_index', 'atom_index_0', 'atom_index_1'])
    atoms['num'] = atom_groups.cumcount() + 2
    atoms = atoms.drop(['d_c'], axis=1)
    atoms = atoms[atoms['num'] < n_atoms]

    atoms = atoms.set_index(['molecule_index', 'atom_index_0', 'atom_index_1', 'num']).unstack()
    atoms.columns = [f'{col[0]}_{col[1]}' for col in atoms.columns]
    atoms = atoms.reset_index()

    # downcast back to int8
    for col in atoms.columns:
        if col.startswith('atom_'):
            atoms[col] = atoms[col].fillna(0).astype('int8')

    atoms['molecule_index'] = atoms['molecule_index'].astype('int32')
    full = add_atoms(base, atoms)
    add_distances(full)

    full.sort_values('id', inplace=True)

    return full

#Función para tomar un DataFrame df y un número máximo de átomos n_atoms para construir un nuevo DataFrame con las características seleccionadas.
def take_n_atoms(df, n_atoms, four_start=4):
    labels = []
    for i in range(2, n_atoms):
        label = f'atom_{i}'
        labels.append(label)

    for i in range(n_atoms):
        num = min(i, 4) if i < four_start else 4
        for j in range(num):
            labels.append(f'd_{i}_{j}')
    if 'scalar_coupling_constant' in df:
        labels.append('scalar_coupling_constant')
    return df[labels]

# %%time
full = build_couple_dataframe(train_csv, structures_csv, '1JHN', n_atoms=10)
# print(full.shape)

full.columns

df = take_n_atoms(full, 7)
# LightGBM performs better with 0-s then with NaN-s
df = df.fillna(0)
df.columns

X_data = df.drop(['scalar_coupling_constant'], axis=1).values.astype('float32')
y_data = df['scalar_coupling_constant'].values.astype('float32')

X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, test_size=0.2, random_state=128)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

# Asumiendo que Xts y yts son X_val y y_val aquí
Xts = X_val
yts = y_val
print("xtss", Xts)
print("ytss", yts)
# # Guardar datos de entrada de prueba
df = pd.DataFrame(Xts)
df.to_csv('test_data_input.csv', index=False, header=False)
# Xts.to_csv("test_data_input.csv", index=False, header=False)

# Guardar etiquetas de prueba en un archivo separado
pd.DataFrame(yts, columns=['scalar_coupling_constant']).to_csv("test_data_target.csv", index=False, header=False)


xtss [[6.        6.        7.        ... 2.3477147 1.3518296 3.6695251]
 [1.        6.        6.        ... 2.646279  2.841643  1.507401 ]
 [1.        6.        6.        ... 2.3082168 2.4148152 1.3393006]
 ...
 [1.        6.        6.        ... 2.9742188 3.7655346 2.208652 ]
 [1.        6.        1.        ... 2.4726589 2.6054046 1.355596 ]
 [7.        6.        6.        ... 2.5813415 3.6075819 1.37785  ]]
ytss [58.8372 46.1204 51.7708 ... 47.131  44.7212 70.6576]


# train model

In [38]:
LGB_PARAMS = {
    'objective': 'regression',
    'metric': 'mae',
    'verbosity': -1,
    'boosting_type': 'gbdt',
    'learning_rate': 0.2,
    'num_leaves': 128,
    'min_child_samples': 79,
    'max_depth': 9,
    'subsample_freq': 1,
    'subsample': 0.9,
    'bagging_seed': 11,
    'reg_alpha': 0.1,
    'reg_lambda': 0.3,
    'colsample_bytree': 1.0
}
#Entrenamiento del modelo de regresión con el algoritmo LGBMRegressor
model = LGBMRegressor(**LGB_PARAMS, n_estimators=1500, n_jobs = -1)
model.fit(X_train, y_train,
        eval_set=[(X_train, y_train), (X_val, y_val)], eval_metric='mae',
        verbose=True, early_stopping_rounds=200)

y_pred = model.predict(X_val)
np.log(mean_absolute_error(y_val, y_pred))

[1]	training's l1: 6.9934	valid_1's l1: 7.03724
Training until validation scores don't improve for 200 rounds
[2]	training's l1: 5.63422	valid_1's l1: 5.67267
[3]	training's l1: 4.54984	valid_1's l1: 4.58309
[4]	training's l1: 3.68512	valid_1's l1: 3.71962
[5]	training's l1: 2.99759	valid_1's l1: 3.02846
[6]	training's l1: 2.44884	valid_1's l1: 2.47529
[7]	training's l1: 2.01502	valid_1's l1: 2.03967
[8]	training's l1: 1.67426	valid_1's l1: 1.69845
[9]	training's l1: 1.41013	valid_1's l1: 1.4374
[10]	training's l1: 1.20353	valid_1's l1: 1.23466
[11]	training's l1: 1.04547	valid_1's l1: 1.08357
[12]	training's l1: 0.927371	valid_1's l1: 0.972783
[13]	training's l1: 0.843122	valid_1's l1: 0.893377
[14]	training's l1: 0.781922	valid_1's l1: 0.837053
[15]	training's l1: 0.732584	valid_1's l1: 0.794495
[16]	training's l1: 0.696755	valid_1's l1: 0.765574
[17]	training's l1: 0.667968	valid_1's l1: 0.743459
[18]	training's l1: 0.647895	valid_1's l1: 0.726721
[19]	training's l1: 0.62993	valid_1

-0.8118539633148267

In [39]:
print ("train score", model.score(X_train, y_train))
print ("test score ", model.score(Xts, yts))

train score 0.9998965653583494
test score  0.9948158850428823


# save model

In [40]:
with open("model.pkl", "wb") as f:
    pickle.dump(model, f)

# load model and test data from files

In [41]:
with open('model.pkl', 'rb') as f:
    _m = pickle.load(f)
    
_Xts = pd.read_csv("test_data_input.csv").values
_yts = pd.read_csv("test_data_target.csv").values


In [42]:
_m.predict(_Xts)

array([45.81137918, 51.51139976, 57.36594091, ..., 46.9721076 ,
       45.15413042, 69.85106785])

In [43]:
_m.score(_Xts, _yts)

0.994816279226156